In [ ]:
from Parameters import ModelParams
from Model_Data_Loader import ModelDataLoader
from Star_Parameters_Class import StarParams
from Mass_Loss_Class import MassLoss
from Fractionation_Class import Fractionation
from Misc_Class import Misc

import numpy as np
import pandas as pd
import json

In [ ]:
# ---------- Initialize parameters and classes ----------
params          = ModelParams()
# set desired outflow mode: 'HHe', 'H2O', or 'HHe_H2O'
# e.g., for pure H/He (no fractionation):
# params.update_param('outflow_mode','HHe')
# for pure water steam:
# params.update_param('outflow_mode','H2O')
# for mixed H/He + H2O:
# params.update_param('outflow_mode','HHe_H2O')

mearth          = params.mearth
rearth          = params.rearth
mass_loss       = MassLoss(params)
fractionation   = Fractionation(params)
star_params     = StarParams(params)
misc            = Misc(params)
loader          = ModelDataLoader('/Users/mvalatsou/PhD/Repos/MR_perplex/OUTPUT/', params)

# ---------- Load data ----------
### option 1
mass, radius, Teq = loader.load_single_ddat_file('3HHe_90H2O_subNeptune.ddat')

### option 2
# with open("planet_params.json", "r") as file:
#     planet_params = json.load(file)
# selected = "TOI-431 d"                 # <------------ change the planet here
# if selected in planet_params:
#     params.update_param('FEUV', planet_params[selected]['FEUV'])
# else:
#     print(f"Make sure the planet you selected exists in the list! Check the spelling.")
# radius = np.array([planet_params[selected]["radius"] * rearth]) # cm
# mass = np.array([planet_params[selected]["mass"] * mearth]) # g
# Teq = np.array([planet_params[selected]["teq"]]) # K

# ---------- Find unique temperatures ----------
unique_Teqs = np.unique(Teq)
print("Unique Teq values found:", unique_Teqs)

all_flux_results = []

for Teq_val in unique_Teqs:
    star_params.update_param("Teq", Teq_val)
    Fbol = star_params.get_Fbol_from_Teq()
    # ---------- Range of EUV flux values ----------
    flux_range = star_params.get_FEUV_range_any_age()   # if option 1 from load data
    # flux_range = [params.get_param('FEUV')]             # if option 2 from load data (.json file)

    mask = np.isclose(Teq, Teq_val)
    mass_group = mass[mask]
    radius_group = radius[mask]
    teq_group = Teq[mask]
    print(f"\nProcessing group with Teq = {Teq_val} K and Fbol = {Fbol}")

    # ---------- Execute ----------
    for flux_value in flux_range:
        print(f"\n Running model with FEUV = {flux_value} erg/cm^2/s")
        params.update_param('FEUV', flux_value)

        # ---------- Mass loss model (mode set by params.outflow_mode) ----------
        mass_loss_results = mass_loss.compute_mass_loss_parameters(mass_group, radius_group, teq_group)
        
        # ---------- Fractionation model (skipped if mode='HHe') ----------
        fractionation_results = fractionation.execute_self_consistent_fractionation(mass_loss_results, mass_loss, misc, params)
        # mdot = fractionation_results[0]['Mdot']
        # print(f"Mdot =", mdot)

        all_flux_results.append({
            'FEUV': flux_value,
            'Teq' : Teq_val,
            'Fbol': Fbol,
            'results': fractionation_results
        })

In [ ]:
# save data for plotting
rows = []

# for flux_data in all_flux_results:
#     for res in flux_data['end_results']:
#         # include FEUV, Teq and Fbol from the outer dict
#         record = {
#             'FEUV':  flux_data['FEUV'],
#             'Teq':   flux_data['Teq'],
#             'Fbol':  flux_data['Fbol'],
#         }
#         record.update(res)
#         rows.append(record)

# df = pd.DataFrame(rows)
# df.to_csv('/Users/mvalatsou/PhD/Repos/MR_perplex/OUTPUT/3H2O_superEarth.csv', index=False)

for entry in all_flux_results:
    feuv = entry['FEUV']
    teq_val = entry['Teq']
    fbol = entry['Fbol']
    for sol in entry['results']:
        row = {'FEUV': feuv, 'Teq': teq_val, 'Fbol': fbol}
        row.update(sol)
        rows.append(row)

df = pd.DataFrame(rows)
output_path = '/Users/mvalatsou/PhD/Repos/MR_perplex/OUTPUT/3HHe_90H2O_subNeptune.csv'
df.to_csv(output_path, index=False)